In [154]:
import pandas as pd
import re
import requests
from dataclasses import dataclass
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
HEADERS={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
firefox_options = Options()
firefox_options.add_argument(f"HEADERS={HEADERS}")
firefox_options.add_argument('-headless')

In [39]:
# Пуляем в бота понравившееся квартиры, он за ними следит

In [40]:

driver = webdriver.Firefox(options=firefox_options,executable_path='../repositories/jobs/ops/geckodriver')
URL = "https://www.cian.ru/sale/flat/280098231/"
# URL = 'https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&f=ASgBAQICAkSSA8YQwMENuv03A0DKCCSCWYRZjt4OFAKQ3g4UAg&p=10&s=104'
driver.get(URL)

/tmp/ipykernel_3298/335986504.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(options=firefox_options,executable_path='../repositories/jobs/ops/geckodriver')


In [41]:
# gone_url = 'https://www.cian.ru/sale/flat/280033231/'

In [42]:
driver.get(URL)

In [176]:
@dataclass
class RealEstateObject:
    url:str
    driver:...
    def __post_init__(self):
        self.parse_page()
        if self.is_on_sale_check():
            self.get_object_data()

    def parse_page(self):
        if self.driver.__module__ == 'selenium.webdriver.firefox.webdriver':
            self.driver.get(self.url)
            page = self.driver.find_element('xpath',"//*").get_attribute("outerHTML")
        else:
            page = self.driver.get(self.url).text
        self.html_data = BeautifulSoup(page,'lxml')
        self.last_update = pd.to_datetime('now')

    def is_on_sale_check(self):
        if (self.last_update - pd.to_datetime('now')).days >= 1:
             self.parse_page()
             self.last_update = pd.to_datetime('now')
        self.is_on_sale = False if 'Объявление снято с публикации' in self.html_data else True
        self.price = float(re.sub('\D','',self.html_data.find('span',{'itemprop':'price'})['content']))
        return self.is_on_sale

    def get_object_data(self):

        self.raw_adress = self.html_data.find('div',{'data-name':'Geo'}).findAll('span')[0]['content']
        self.metro_list = [step.text for step in self.html_data.find('div',{'data-name':'Geo'}).findAll('a')[5:]]
        self.m2 = re.findall('(\d{2,})',self.html_data.find('div',{'data-name':'OfferTitle'}).text)
        self.rooms = self.html_data.find('div',{'data-name':'OfferTitle'}).text[0]
        self.floor = re.findall('(\d{,2})? из',self.html_data.find('div',{'data-name':'ObjectSummaryDescription'}).text)
        self.build_year = re.findall('(\d{4})',self.html_data.find('div',{'data-name':'ObjectSummaryDescription'}).text)
        self.description = self.html_data.find('p',{'itemprop':'description'}).text
        self.adress_items = dict(zip(('Город','Округ','Район','Улица','Дом'),[step.text for step in self.html_data.find('div',{'data-name':'Geo'}).findAll('a')[:5]]))
        self.home_items = {item.findAll('div')[0].text : item.findAll('div')[1].text for item in self.html_data.find('div',{'data-name':'BtiHouseData'}).findAll('div',{'data-name':'Item'})}
        self.img_list = [img['src'] for img in self.html_data.find('div',{'data-name':'GalleryInnerComponent'}).findAll('img') if img['src'].endswith('.jpg')]
    
    def save_figs(url,name):
        img_bytes = requests.get(url).content
        with open(f'{name}.jpg', 'wb') as f:
            f.write(img_bytes)


In [151]:
some_RE = RealEstateObject(url = 'https://www.cian.ru/sale/flat/280098231/',driver=driver)

In [170]:
r_session = requests.Session()

In [177]:
some_RE_rses = RealEstateObject(url = 'https://www.cian.ru/sale/flat/280098231/',driver=r_session)

In [181]:
driver.close()